# Solving the OCP via direct approach with new approach and standard

State equations: forced euler-Lagrange:
$$
\frac{d}{dt}D_2 L = D_1 L + f_L^\mathcal{E}
$$


# imports

In [ ]:
import sympy
import numpy as np
import function_definitions_forced as field_funcs
import helper_funcs as hfct
import pickle
import scipy
import scipy.optimize as opt
import OCP_construction_functions_forced as OCP_funcs
import matplotlib.pyplot as plt
import sympy
import numpy as np

# Parameter definitions

In [ ]:

################################################################################
################################################################################
## Vary here N and alpha, gamma
################################################################################
################################################################################

iterater = 1
N_choice = [50,100,150,200,250,300,350,400,450,500,600][iterater] 
alpha_choice = 1
gamma_choice= 0.5


################################################################################
################################################################################
# End variation
################################################################################
################################################################################


#################################################################
#################################################################
# Give here link 
#################################################################
#################################################################

save_data_file = 'data/pendulum_inversion/reference/pendulum_reference.pkl'

#################################################################
#################################################################

#################################################################
#################################################################


OCP_parameters = {
            # Generic OCP params, always needed
            "T": 3.,   #s
            # "dim_q": 2,  # state dimension will be set by the control Lagrangian
            # "dim_u": 1,  # actuation dimension will be set by the control Lagrangian
            "N":N_choice,     #240, 160, 80, 40, 20
            "q0": sympy.Matrix([0.,0.]),        #(m,1)
            "dq0": sympy.Matrix([0.,0.]),       #(m/s,1/s)
            "qT": sympy.Matrix([0.,sympy.pi]),       #(r,phi) 
            "dqT": sympy.Matrix([0.,0.]),       #(vr,vphi)
            "alpha": sympy.Symbol("alpha"),
            "gamma": sympy.Symbol("gamma"),
            'folder_name': 'pendulum_inversion',
            #specific to model
            "m1":10.,  # kg
            "m2":9.,
            "l":0.5,
            "I":3.2,
            "G": 9.81, #m/s**2
            "Aq": 123.7, #  kg/s^2
            "Adq": 127.7, # kg/s
            "A_u": 0.00052,
            "dim_u":1,
            "variable_names": ["x", "theta"]
        }
OCP_parameters["h"] = sympy.symbols("h")
OCP_parameters["times"] = sympy.Matrix([i*OCP_parameters["h"] for i in range(OCP_parameters["N"]+1)])

# Function definition

In [ ]:
continuous_eq = OCP_funcs.Direct_continuous_generator_forced_L(OCP_parameters,Lagrangian=field_funcs.pendulum_cart_lagrangian,f_L=field_funcs.f_L,running_cost_func=field_funcs.running_cost_pendulum,mayer_func=field_funcs.mayer_term_pendulum,g_func=field_funcs.g_mat_pendulum,I_func=field_funcs.conserved_I_control_pendulum)

# initial guess creation

In [ ]:
#####
# new approach of getting initial values with the new structure of data
#####

alpha_value,beta_value,gamma_value=alpha_choice,gamma_choice,gamma_choice
def dphi_from_radius(radius,grav,Mass):
    return  np.sqrt(grav* Mass/radius**3)
def rescale_stepsize_parameters(parameters, stepsize):
    parameters["h"] = stepsize
    parameters["times"] = np.array([parameters["t_0"] + parameters["h"]*i for i in range(parameters["N"]+1)])
OCP_parameters_np_version = hfct.sympy_to_np_dict(OCP_parameters,alpha_choice,gamma_choice)
parameters = OCP_parameters_np_version 

with open(save_data_file, 'rb') as files:
    initial_guess_data = pickle.load(files)

q_d_start_guess = initial_guess_data["q_d_new"]

vq_d_start_guess = initial_guess_data["v_y_d_new"][:,:2]
lam_d_start_guess = initial_guess_data["lambda_d_new"]
u_d_start_guess = initial_guess_data['u_d_new']
lam_q_start_guess = initial_guess_data["lambda_q_standard"]
lam_v_start_guess = initial_guess_data["lambda_v_standard"]
cs_q =  scipy.interpolate.CubicSpline(np.linspace(0,parameters["T"],len(q_d_start_guess)), q_d_start_guess.reshape([len(q_d_start_guess),2]))
cs_vq =  scipy.interpolate.CubicSpline(np.linspace(0,parameters["T"],len(vq_d_start_guess)), vq_d_start_guess.reshape([len(vq_d_start_guess),2]))
cs_lam = scipy.interpolate.CubicSpline(np.linspace(0,parameters["T"],len(lam_d_start_guess)), lam_d_start_guess.reshape([len(lam_d_start_guess),2]))  
cs_lam_q = scipy.interpolate.CubicSpline(np.linspace(0,parameters["T"],len(lam_q_start_guess)), lam_q_start_guess.reshape([len(lam_q_start_guess),2]))  
cs_lam_v = scipy.interpolate.CubicSpline(np.linspace(0,parameters["T"],len(lam_v_start_guess)), lam_v_start_guess.reshape([len(lam_v_start_guess),2]))  
cs_u= scipy.interpolate.CubicSpline(np.linspace(0,parameters["T"],len(u_d_start_guess)),u_d_start_guess)

U_d_1_use = cs_u(np.array(parameters["times"]) + parameters["gamma"]*parameters["h"]).reshape(len(parameters["times"]),1,1)
U_d_2_use = cs_u(np.array(parameters["times"]) + (1-parameters["gamma"])*parameters["h"]).reshape(len(parameters["times"]),1,1)

q_d_use = cs_q(parameters["times"]).reshape(len(parameters["times"]),2,1)
lambda_d_use = cs_lam(parameters["times"]).reshape(len(parameters["times"]),2,1)
lambda_q_d_use = cs_lam_q(parameters["times"]).reshape(len(parameters["times"]),2,1)
lambda_v_d_use = cs_lam_v(parameters["times"]).reshape(len(parameters["times"]),2,1)
mu_use = initial_guess_data['mu_sol_new']
nu_use =  initial_guess_data['nu_sol_new']
v_q_use = cs_vq(parameters["times"]).reshape(len(parameters["times"]),2,1)

initial_guess= list(mu_use) + list(nu_use)
initial_guess_new = list(mu_use) + list(nu_use)

for tmp in q_d_use:
    initial_guess += list(tmp.flatten())
    initial_guess_new+=list(tmp.flatten())
    #vq guess
for tmp in v_q_use:
    initial_guess += list(tmp.flatten())
#lambda terms
for tmp in lambda_d_use:
    initial_guess_new += list(tmp.flatten())

for tmp in lambda_q_d_use:
    initial_guess += list(tmp.flatten())
for tmp in lambda_v_d_use:
    initial_guess += list(tmp.flatten())

initial_guess += list(U_d_1_use.flatten()) 
initial_guess += list(U_d_2_use.flatten()) 
initial_guess_new += list(U_d_1_use.flatten()) 
initial_guess_new += list(U_d_2_use.flatten()) 

# needed because v_y creates sympy objects...
initial_guess=np.array(initial_guess,dtype='float')
initial_guess_new=np.array(initial_guess_new,dtype='float')



# Evolution via state eq

In [ ]:
discrete_equations = OCP_funcs.discrete_standard_direct_eq_generator_forced_L(continuous_eq)

# lambdify the equations for root finding

In [ ]:
#KKT direct approach
standard_direct_midpoint_KKT = discrete_equations.calc_KKT(OCP_parameters_np_version)
lambdified_KKT =  sympy.lambdify(standard_direct_midpoint_KKT[1],standard_direct_midpoint_KKT[0].subs(discrete_equations.h,OCP_parameters_np_version["h"]).subs(discrete_equations.cont_equations.parameters["alpha"],alpha_choice).subs(discrete_equations.cont_equations.parameters["gamma"],gamma_choice)) 
lambdified_KKT_eval = lambda x :lambdified_KKT(*x)


In [ ]:
#KKT direct approach with new Lagrangian
standard_direct_KKT_new = discrete_equations.calc_KKT_new(OCP_parameters_np_version)

lambdified_KKT_new =  sympy.lambdify(standard_direct_KKT_new[1],standard_direct_KKT_new[0].subs(discrete_equations.h,OCP_parameters_np_version["h"]).subs(discrete_equations.cont_equations.parameters["alpha"],alpha_choice).subs(discrete_equations.cont_equations.parameters["gamma"],gamma_choice))

lambdified_KKT_new_eval = lambda x :lambdified_KKT_new(*x)


# Post-processing of initial guess data

In [ ]:
v_vec = discrete_equations.cont_equations.v_y_as_p_y()
v_vec_lambdified = sympy.lambdify(sympy.flatten(discrete_equations.cont_equations.q)+sympy.flatten(discrete_equations.cont_equations.lamq)+sympy.flatten(discrete_equations.cont_equations.pq)+sympy.flatten(discrete_equations.cont_equations.plam),v_vec)

v_eval = lambda q,lam,p_y: v_vec_lambdified(*q,*lam,*p_y)

In [ ]:
p_y_d_use = np.array(sympy.Matrix([sympy.flatten(tmp) for tmp in discrete_equations.p_v_d_from_y_d(q_d_use[:,:,0],lambda_d_use[:,:,0],U_d_1_use[:,:,0],U_d_2_use[:,:,0],OCP_parameters_np_version)]),dtype=float)
v_y_d_use = []
for tmp1,tmp2,tmp3 in zip(q_d_use[:,:,0],lambda_d_use[:,:,0],p_y_d_use):
    v_y_d_use.append(sympy.flatten(v_eval(tmp1,tmp2,tmp3)))
v_y_d_use = np.array(v_y_d_use) 


In [ ]:
# comparison H, u_d, I calc

u_vec_comparison = []
for i in range(len(q_d_use)):
    u_vec_comparison.append(discrete_equations.cont_equations.u_eval_from_new(q_d_use[i],lambda_d_use[i],p_y_d_use[i][:2],p_y_d_use[i][2:],OCP_parameters))

u_vec_comparison = np.array(u_vec_comparison)

H_control_comparison = []
for i in range(len(q_d_use)):
    H_control_comparison.append(discrete_equations.cont_equations.new_control_H_eval(q_d_use[i],lambda_d_use[i],p_y_d_use[i][:2],p_y_d_use[i][2:],u_vec_comparison[i],OCP_parameters_np_version))
H_control_comparison = np.array(H_control_comparison)

# Root finding of KKT conditions

In [ ]:
# #standard direct approach
direct_result_KKT = opt.root(lambdified_KKT_eval,x0=initial_guess,method="lm")
direct_result_KKT = opt.root(lambdified_KKT_eval,x0=direct_result_KKT.x,method="lm")

In [ ]:
#new lagrangian approach
direct_result_KKT_new = opt.root(lambdified_KKT_new_eval,method='lm',x0=initial_guess_new,options={'ftol': 1e-13,'xtol':1e-13})


In [ ]:
direct_result_KKT

In [ ]:
direct_result_KKT_new

# Post-processing

In [ ]:
reshape_params = discrete_equations.cont_equations.parameters
reshape_N = reshape_params["N"]
reshape_dim_q = reshape_params["dim_q"]
reshape_dim_u = reshape_params["dim_u"]
mu_KKT,nu_KKT = direct_result_KKT.x[:2*reshape_dim_q].reshape(2,reshape_dim_q)
q_d_KKT = direct_result_KKT.x[2*reshape_dim_q:2*reshape_dim_q + reshape_dim_q * (reshape_N+1)].reshape(reshape_N + 1,reshape_dim_q)
vq_d_KKT = direct_result_KKT.x[2*reshape_dim_q + reshape_dim_q * (reshape_N+1):2*reshape_dim_q + 2*reshape_dim_q * (reshape_N+1)].reshape(reshape_N + 1,reshape_dim_q)
lamq_d_KKT = direct_result_KKT.x[2*reshape_dim_q + 2*reshape_dim_q * (reshape_N+1):2*reshape_dim_q + 3*reshape_dim_q * (reshape_N+1)].reshape(reshape_N + 1,reshape_dim_q)
lamv_d_KKT = direct_result_KKT.x[2*reshape_dim_q + 3*reshape_dim_q * (reshape_N+1):2*reshape_dim_q + 4*reshape_dim_q * (reshape_N+1)].reshape(reshape_N + 1,reshape_dim_q)
U1_d_KKT = direct_result_KKT.x[2*reshape_dim_q + 4*reshape_dim_q * (reshape_N+1):2*reshape_dim_q + 4*reshape_dim_q * (reshape_N+1) + (reshape_N+1)*reshape_dim_u ].reshape(reshape_N + 1,reshape_params["dim_u"])
U2_d_KKT = direct_result_KKT.x[2*reshape_dim_q + 4*reshape_dim_q * (reshape_N+1)+ (reshape_N+1)*reshape_dim_u:].reshape(reshape_N + 1,reshape_params["dim_u"])

u_vec_KKT = []
for i in range(len(q_d_KKT)):
    u_vec_KKT.append(discrete_equations.cont_equations.u_Pontryagin_eval(q_d_KKT[i],vq_d_KKT[i],lamq_d_KKT[i],lamv_d_KKT[i],OCP_parameters_np_version))
u_vec_KKT= np.array(u_vec_KKT)
H_Pontry_KKT = []
for i in range(len(q_d_KKT)):
    H_Pontry_KKT.append(discrete_equations.cont_equations.Pontryagin_H_eval(q_d_KKT[i],vq_d_KKT[i],lamq_d_KKT[i],lamv_d_KKT[i],u_vec_KKT[i],OCP_parameters_np_version))
H_Pontry_KKT = np.array(H_Pontry_KKT)    



In [ ]:
reshape_params = discrete_equations.cont_equations.parameters
reshape_N = reshape_params["N"]
reshape_dim_q = reshape_params["dim_q"]
reshape_dim_u = reshape_params["dim_u"]
mu_KKT_new,nu_KKT_new = direct_result_KKT_new.x[:2*reshape_dim_q].reshape(2,reshape_dim_q)
q_d_KKT_new = direct_result_KKT_new.x[2*reshape_dim_q:2*reshape_dim_q + reshape_dim_q * (reshape_N+1)].reshape(reshape_N + 1,reshape_dim_q)
lam_d_KKT_new = direct_result_KKT_new.x[2*reshape_dim_q + reshape_dim_q * (reshape_N+1):2*reshape_dim_q + 2*reshape_dim_q * (reshape_N+1)].reshape(reshape_N + 1,reshape_dim_q)

U1_d_KKT_new = direct_result_KKT_new.x[2*reshape_dim_q + 2*reshape_dim_q * (reshape_N+1):2*reshape_dim_q + 2*reshape_dim_q * (reshape_N+1) + (reshape_N+1)*reshape_dim_u].reshape(reshape_N + 1,reshape_params["dim_u"])
U2_d_KKT_new = direct_result_KKT_new.x[2*reshape_dim_q + 2*reshape_dim_q * (reshape_N+1) + (reshape_N+1)*reshape_dim_u:].reshape(reshape_N + 1,reshape_params["dim_u"])

p_y_d_new = np.array([sympy.flatten(tmp) for tmp in discrete_equations.p_v_d_from_y_d(q_d_KKT_new,lam_d_KKT_new,U1_d_KKT_new,U2_d_KKT_new,OCP_parameters_np_version)])



u_vec_KKT_new = []
for i in range(len(q_d_KKT_new)):
    u_vec_KKT_new.append(discrete_equations.cont_equations.u_eval_from_new(q_d_KKT_new[i],lam_d_KKT_new[i],p_y_d_new[i][:2],p_y_d_new[i][2:],OCP_parameters))

u_vec_KKT_new = np.array(u_vec_KKT_new)

H_control_KKT_new = []
for i in range(len(q_d_KKT_new)):
    H_control_KKT_new.append(discrete_equations.cont_equations.new_control_H_eval(q_d_KKT_new[i],lam_d_KKT_new[i],p_y_d_new[i][:2],p_y_d_new[i][2:],u_vec_KKT_new[i],OCP_parameters_np_version))
H_control_KKT_new = np.array(H_control_KKT_new)  



In [ ]:
v_y_d_new = []
for tmp1,tmp2,tmp3 in zip(q_d_KKT_new,lam_d_KKT_new,p_y_d_new):
    v_y_d_new.append(sympy.flatten(v_eval(tmp1,tmp2,tmp3)))
v_y_d_new = np.array(v_y_d_new) 


if discrete_equations.cont_equations.I_func is not None:
    I_func_lambdified = sympy.lambdify(sympy.flatten(discrete_equations.all_vars_new_approach),discrete_equations.cont_equations.I_func)
    I_new_evo = []
    for tmpq,tmplam,tmpvy,tmpu in zip(q_d_KKT_new,lam_d_KKT_new,v_y_d_new,u_vec_KKT_new):
        I_new_evo.append(np.array(I_func_lambdified(*tmpq,*tmpvy[:2],*tmplam,*(tmpvy[2:]),*tmpu),float).flatten())
    I_new_evo=np.array(I_new_evo).flatten()

    

In [ ]:
xi_calc_standard = np.array([continuous_eq.calc_xi_from_standard_variables(tmpq,tmplamq) for tmpq,tmplamq in zip(q_d_KKT,lamv_d_KKT)])
continuous_eq.calc_v_xi_from_standard_variables()
v_xi_calc_standard = []
vxi_calc_test = []
for i in range(len(q_d_KKT)):
    v_xi_calc_standard.append(continuous_eq.eval_v_xi(q_d_KKT[i],vq_d_KKT[i],lamq_d_KKT[i],lamv_d_KKT[i],xi_calc_standard[i],u_vec_KKT[i]))
v_xi_calc_standard = np.array(v_xi_calc_standard)


p_y_d_standard = np.array([sympy.flatten(tmp) for tmp in discrete_equations.p_v_d_from_y_d(q_d_KKT,xi_calc_standard,U1_d_KKT,U2_d_KKT,OCP_parameters_np_version)])
H_control_standard = []
for i in range(len(q_d_KKT)):
    H_control_standard.append(discrete_equations.cont_equations.new_control_H_eval(q_d_KKT[i],xi_calc_standard[i],p_y_d_standard[i][:2],p_y_d_standard[i][2:],u_vec_KKT[i],OCP_parameters_np_version))
H_control_standard = np.array(H_control_standard)  



I_standard_evo = []
for tmpq,tmplam,tmpvq,tmpvlam,tmpu in zip(q_d_KKT,xi_calc_standard,vq_d_KKT,v_xi_calc_standard,u_vec_KKT):
    I_standard_evo.append(np.array(I_func_lambdified(*tmpq,*tmpvq,*tmplam,*tmpvlam,*tmpu),float)[0])
I_standard_evo = np.array(I_standard_evo).flatten()

# plt.plot(xi_calc_standard.transpose()[0],label=r'$\xi_x$ from standard')
# plt.plot(lam_d_KKT_new.transpose()[0],'--',label=r'$\xi_x$ from new')
# plt.legend()
# plt.show()
# plt.plot(xi_calc_standard.transpose()[1],label=r'$\xi_\theta$ from standard')
# plt.plot(lam_d_KKT_new.transpose()[1],'--',label=r'$\xi_\theta$ from new')
# plt.legend()
# plt.show()
# plt.plot(v_xi_calc_standard.transpose()[0],label='standard')
# plt.plot(v_y_d_new.transpose()[2],label=r'$v_{\xi_x}$new ')
# plt.legend()


# plot results

In [ ]:
q_d_standard_KKT_sol_cart = []
q_d_new_KKT_sol_cart = []
q_d_standard_KKT_sol_cart = q_d_KKT
q_d_new_KKT_sol_cart = q_d_KKT_new
q_d_standard_KKT_sol_cart = np.array(q_d_standard_KKT_sol_cart)
q_d_new_KKT_sol_cart = np.array(q_d_new_KKT_sol_cart)


plt.plot(q_d_use[:,0].flatten(),q_d_use[:,1].flatten(),'*',label='xy sol')
plt.plot(*q_d_standard_KKT_sol_cart.transpose(),label='standard x,y evo')
plt.plot(*q_d_new_KKT_sol_cart.transpose(),'--',label='new x,y evo')
plt.legend()
plt.xlabel('x')
plt.ylabel('y')
plt.tight_layout()

plt.show()

plt.plot(OCP_parameters_np_version["times"][:-1],xi_calc_standard.transpose()[0][:-1],label='standard xi_r')

plt.plot(OCP_parameters_np_version["times"],lam_d_KKT_new.transpose()[0],'--',label='new approach lam_r')
plt.legend()
plt.xlabel('N step')
plt.ylabel('lambda_r')
plt.tight_layout()

plt.show()
plt.plot(OCP_parameters_np_version["times"][:-1],xi_calc_standard.transpose()[1][:-1],label='standard lamv_phi')
plt.plot(OCP_parameters_np_version["times"],lam_d_KKT_new.transpose()[1],'--',label='new aproach lam_phi')
plt.legend()
plt.xlabel('N step')
plt.ylabel('lambda_phi')
plt.tight_layout()


plt.show()
plt.plot(U1_d_KKT[:-1],label='standard')
plt.plot(U1_d_KKT_new[:-1],'x',label='new')
plt.plot(U_d_1_use[:-1,0,0],'--',label='comparison')
plt.legend()
plt.xlabel('N step')
plt.ylabel('U_1')
plt.tight_layout()


plt.show()

plt.plot(U2_d_KKT[:-1],label='standard')
plt.plot(U2_d_KKT_new[:-1],'x',label='new')
plt.plot(U_d_2_use[:-1,0,0],'--',label='comparison')
plt.legend()
plt.xlabel('N step')
plt.ylabel('U_2')
plt.tight_layout()


plt.show()
plt.plot(U_d_2_use[:-1,0,0],'--',label='comparison')
plt.plot(u_vec_KKT_new[:,0],label='new ud')
plt.plot(u_vec_KKT[:,0],label='standard ud')
plt.xlabel('N step')
plt.ylabel('u_d')
plt.legend()


plt.show()
plt.plot(np.array(I_new_evo)[:-2],'X-',label="I new")

plt.legend()
plt.xlabel('N step')
plt.ylabel('I')
plt.tight_layout()

plt.show()
plt.plot(-(H_Pontry_KKT[:-1]), label='Pontryagin H')
plt.plot((H_control_KKT_new[:-1]),'--',label='new control H')
plt.legend()

plt.xlabel('N step')
plt.ylabel('H')


# Saving data

In [ ]:
from pathlib import Path 

if not direct_result_KKT_new.success:
    print('did not find a solution in control dependent case, not storing the result')
else:
    storage_dict = dict()
    storage_dict["parameters"] = OCP_parameters_np_version
    storage_dict["q_d_new"] = q_d_KKT_new
    storage_dict["lambda_d_new"] = lam_d_KKT_new
    storage_dict["p_y_d_new"] = np.array(p_y_d_new,dtype=float)
    storage_dict["U1_d_new"] = np.array(U1_d_KKT_new,dtype=float)
    storage_dict["U2_d_new"] = np.array(U2_d_KKT_new,dtype=float)
    storage_dict["u_d_new"] = np.array(u_vec_KKT_new,dtype=float)
    storage_dict["I_d_new"] = np.array(I_new_evo,dtype=float)
    storage_dict["H_control_new"] = np.array(H_control_KKT_new,dtype=float)
    storage_dict["v_y_d_new"] = np.array(v_y_d_new,dtype=float)
    storage_dict["q_d_standard"] =   q_d_KKT
    storage_dict["lambda_q_standard"] = lamq_d_KKT
    storage_dict["lambda_v_standard"] = lamv_d_KKT
    storage_dict["vq_d_standard"] = vq_d_KKT
    storage_dict["U1_d_standard"] = U1_d_KKT
    storage_dict["U2_d_standard"] = U2_d_KKT
    storage_dict["u_d_standard"] = u_vec_KKT
    storage_dict["H_pontry_standard"] = H_Pontry_KKT
    storage_dict["mu_sol_new"] = mu_KKT_new 
    storage_dict["nu_sol_new"] = nu_KKT_new 
    storage_dict["xi_calc_standard"] = xi_calc_standard
    storage_dict["v_xi_calc_standard"] = v_xi_calc_standard
    storage_dict["I_standard_evo"] = I_standard_evo
    storage_dict["p_y_d_standard"] = p_y_d_standard


    dirpath = "data/" + OCP_parameters_np_version["folder_name"]+'smallerqd'+"/data_"+"a=" + str(OCP_parameters_np_version["alpha"])  +"g=" + str(OCP_parameters_np_version["gamma"])
    Path(dirpath).mkdir(parents=True, exist_ok=True)

    file_name = dirpath+"/data_" +"a=" + str(OCP_parameters_np_version["alpha"])  +"g=" + str(OCP_parameters_np_version["gamma"])  +"N=" + str(OCP_parameters_np_version["N"]) + ".pkl"
    with open(file_name, 'wb') as ffile:
        pickle.dump(storage_dict, ffile)
        ffile.close()